In [1]:
inputExample = open('day20-example.txt', 'r').read().split('\n\n')
inputReal = open('day20.txt', 'r').read().split('\n\n')

import re

In [2]:
class Tile:
    def __init__(self, input):
        lines = input.split('\n')
        m = re.match('^Tile ([0-9]+):', lines[0])
        self.id = int(m.group(1))
        del lines[0]

        if lines[-1] == '':
            del lines[-1]

        # print(self.tile_id, lines)

        self.lines = lines
        rightEdge = "".join([x[-1] for x in lines])
        leftEdge = "".join([x[0] for x in lines[::-1]])
        self.edges = [lines[0], rightEdge, lines[-1][::-1], leftEdge]

        self.left = None
        self.right = None
        self.top = None
        self.bottom = None

        self.rotated = 0
        self.flipped = False

        self.coordinates = ('?', '?')

        # print(f"Created tile {self.id} edges", self.edges)


    def __str__(self):
        return f"<Tile {self.id} {self.edges} {self.coordinates}>"

    def __repr__(self):
        return self.__str__()

    def connect(self, connecting_tile: '__class__', this_edge: int, other_edge: int, flip: bool):
        """Connects two tiles together, flipping/rotating the other tile as appropriate"""
        if flip:
            connecting_tile.flip()
            if other_edge == 1:
                other_edge = 3
            elif other_edge == 3:
                other_edge = 1

        while (this_edge + 2) % 4 != other_edge:
            # print("Rotating tile", connecting_tile.id, "-", connecting_tile.edges, end=" ")

            # rotate clockwise
            other_edge = (other_edge + 1) % 4
            connecting_tile.rotate()
            # popped = connecting_tile.edges.pop(-1)
            # connecting_tile.edges.insert(0, popped)
            # connecting_tile.rotated = (connecting_tile.rotated + 1) % 4

            # print(connecting_tile.edges)

        # print("Attempting to connect tile", self.id, "edge", this_edge, "with tile", connecting_tile.id, "edge", other_edge)
        # print(self.edges[this_edge], connecting_tile.edges[other_edge])
        if this_edge == 0 and other_edge == 2:
            self.top = connecting_tile
            connecting_tile.bottom = self
        elif this_edge == 1 and other_edge == 3:
            self.right = connecting_tile
            connecting_tile.left = self
        elif this_edge == 2 and other_edge == 0:
            self.bottom = connecting_tile
            connecting_tile.top = self
        elif this_edge == 3 and other_edge == 1:
            self.left = connecting_tile
            connecting_tile.right = self
        else:
            raise Exception("Unknown edge pair to connect!")


    def flip(self):
        """Horizontally flips a tile"""
        self.flipped = not self.flipped

        newEdges = [
            self.edges[0][::-1],
            self.edges[3][::-1],
            self.edges[2][::-1],
            self.edges[1][::-1]
        ]
        # print("Flipping tile", self.id, "-", self.edges, newEdges)
        self.edges = newEdges

    def rotate(self):
        popped = self.edges.pop(-1)
        self.edges.insert(0, popped)
        self.rotated = (self.rotated + 1) % 4

    def unconnected_edges(self) -> list:
        """Returns a list of unconnected edges, with the connected edges substituted for a non-matching placeholder"""
        return [self.edges[0] if self.top is None else "connected",
                  self.edges[1] if self.right is None else "connected",
                  self.edges[2] if self.bottom is None else "connected",
                  self.edges[3] if self.left is None else "connected"]

    def get_image(self):
        workingSet = [row[1:-1] for row in self.lines[1:-1]]
        return self.apply_changes(workingSet)

    def get_full_image(self):
        return self.apply_changes(self.lines)

    def apply_changes(self, working_set):
        if self.flipped:
            working_set = [r[::-1] for r in working_set]

        if self.rotated == 0:
            # +ve X, +ve Y
            return working_set

        if self.rotated == 1:
            # -ve Y, +ve X
            img = []
            for x in range(0, len(working_set[0])):
                line = ""
                for y in range(len(working_set) - 1, -1, -1):
                    line += working_set[y][x]
                img.append(line)
            return img

        if self.rotated == 2:
            # -ve X, -ve Y
            img = []
            for y in range(len(working_set) - 1, -1, -1):
                line = ""
                for x in range(len(working_set[0]) - 1, -1, -1):
                    line += working_set[y][x]
                img.append(line)
            return img

        if self.rotated == 3:
            # +ve Y, -ve X
            img = []
            for x in range(len(working_set[0]) - 1, -1, -1):
                line = ""
                for y in range(0, len(working_set)):
                    line += working_set[y][x]
                img.append(line)
            return img


In [3]:
def part1(input):
    unconnectedTiles = []
    for tile in input:
        unconnectedTiles.append(Tile(tile))

    imageTiles = [unconnectedTiles[0]]
    del unconnectedTiles[0]

    changed = True
    while len(unconnectedTiles) > 0 and changed:
        changed = False
        breakLoop = False
        for tile in imageTiles:
            for unconnectedImageEdge in tile.unconnected_edges():
                unconnectedImageEdgeReverse = unconnectedImageEdge[::-1]

                for potentiallyConnectingTile in unconnectedTiles:
                    if unconnectedImageEdgeReverse in potentiallyConnectingTile.edges:
                        thisEdgeIdx = tile.edges.index(unconnectedImageEdge)
                        connectingEdgeIdx = potentiallyConnectingTile.edges.index(unconnectedImageEdgeReverse)
                        # print("Found connection tile", tile.id, "edge", thisEdgeIdx, "== tile", potentiallyConnectingTile.id, "reverse edge", connectingEdgeIdx)

                        tile.connect(potentiallyConnectingTile, thisEdgeIdx, connectingEdgeIdx, False)

                        unconnectedTiles.remove(potentiallyConnectingTile)
                        imageTiles.append(potentiallyConnectingTile)

                        changed = True
                        breakLoop = True
                        break # potentiallyConnectingTile in unconnectedTiles

                if breakLoop:
                    break # unconnectedImageEdge in tile.unconnectedEdges()

                for potentiallyConnectingTile in unconnectedTiles:
                    if unconnectedImageEdge in potentiallyConnectingTile.edges:
                        thisEdgeIdx = tile.edges.index(unconnectedImageEdge)
                        connectingEdgeIdx = potentiallyConnectingTile.edges.index(unconnectedImageEdge)
                        # print("Found connection tile", tile.id, "edge", thisEdgeIdx, "== tile", potentiallyConnectingTile.id, "edge", connectingEdgeIdx)

                        tile.connect(potentiallyConnectingTile, thisEdgeIdx, connectingEdgeIdx, True)

                        unconnectedTiles.remove(potentiallyConnectingTile)
                        imageTiles.append(potentiallyConnectingTile)

                        changed = True
                        breakLoop = True
                        break # potentiallyConnectingTile in unconnectedTiles

                if breakLoop:
                    break # unconnectedImageEdge in tile.unconnectedEdges()
            if breakLoop:
                break # tile in imageTiles

    # print("unconnected", len(unconnectedTiles), "connected", len(imageTiles))

    # for t in imageTiles:
    #     print("Tile", t.id)
    #     print("  top", t.top.id if t.top is not None else '--', t.edges[0])
    #     print("  right", t.right.id if t.right is not None else '--', t.edges[1])
    #     print("  bottom", t.bottom.id if t.bottom is not None else '--', t.edges[2])
    #     print("  left", t.left.id if t.left is not None else '--', t.edges[3])
    #
    # for t in unconnectedTiles:
    #     print("Tile", t.id)
    #     print("  UNCONNECTED")

    def find_corners(tile: Tile, from_tile_id: int, this_tile_coords):
        # print("Traversing", tile.id, "from", from_tile_id)
        tile.coordinates = this_tile_coords

        if tile.top and tile.top.id != from_tile_id:
            find_corners(tile.top, tile.id, (this_tile_coords[0], this_tile_coords[1]-1))
        if tile.right and tile.right.id != from_tile_id:
            find_corners(tile.right, tile.id, (this_tile_coords[0]+1, this_tile_coords[1]))
        if tile.bottom and tile.bottom.id != from_tile_id:
            find_corners(tile.bottom, tile.id, (this_tile_coords[0], this_tile_coords[1]+1))
        if tile.left and tile.left.id != from_tile_id:
            find_corners(tile.left, tile.id, (this_tile_coords[0]-1, this_tile_coords[1]))

    find_corners(imageTiles[0], None, (0,0))

    maxX = max([t.coordinates[0] for t in imageTiles])
    minX = min([t.coordinates[0] for t in imageTiles])
    maxY = max([t.coordinates[1] for t in imageTiles])
    minY = min([t.coordinates[1] for t in imageTiles])

    corners = [(minX, minY),(minX, maxY),(maxX, minY),(maxX, maxY)]

    result = 1
    for t in [t.id for t in imageTiles if t.coordinates in corners]:
        result = result * t

    print("part 1", result)
    return imageTiles

In [4]:
_ = part1(inputExample)

part 1 20899048083289


In [5]:
_ = part1(inputReal)

part 1 15006909892229


In [90]:
def part2(imageTiles):
    # t = Tile('Tile 1:\n'+'#-t>#\n'+'>123-\n'+'l456r\n'+'-789>\n'+'#>b-#')

    # for y in range(len(img)):
    #     print(img[y])

    maxX = max([t.coordinates[0] for t in imageTiles])
    minX = min([t.coordinates[0] for t in imageTiles])
    maxY = max([t.coordinates[1] for t in imageTiles])
    minY = min([t.coordinates[1] for t in imageTiles])

    tySize = len(imageTiles[0].lines)-2

    image = []
    for iy in range(minY, maxY +1):
        for ty in range(0, tySize):
            line = ""
            for ix in range(minX, maxX +1):
                line += ([t for t in imageTiles if t.coordinates == (ix, iy)].pop().get_image()[ty])
            image.append(line)

    image.insert(0, "Tile 0:")
    t = Tile('\n'.join(image))

    sea_monster = [
        '                  # ',
        '#    ##    ##    ###',
        ' #  #  #  #  #  #   ']

    sea_monster_regex = [
        '(..................)#(.)', # 2
        '#(....)##(....)##(....)###', # 3
        '(.)#(..)#(..)#(..)#(..)#(..)#(...)'] #7

    for r in range(0, 8):
        f = locate_sea_monster(sea_monster_regex, t)
        if f:
            break

        t.rotate()

        if r == 3:
            t.flip()

    pass


def locate_sea_monster(sea_monster_regex: list, t: Tile) -> bool:
    full_image = t.get_full_image()
    ret_val = False
    for i in range(1, len(full_image) - 1):
        m = re.search(sea_monster_regex[1], full_image[i])
        if not m:
            # print(i, "no match", full_image[i])
            pass
        else:
            print(i, "possible sea monster:", re.sub(sea_monster_regex[1], r'O\1OO\2OO\3OOO', full_image[i]))

            if re.match(sea_monster_regex[0], full_image[i - 1][m.start():]) and re.match(sea_monster_regex[2], full_image[i + 1][m.start():]):
                print('  SEA MONSTER CONFIRMED!')

                prefix = '^(' + ('.' * m.start()) + ')'

                sm_top = prefix + sea_monster_regex[0]
                sm_middle = prefix + sea_monster_regex[1]
                sm_bottom = prefix + sea_monster_regex[2]

                print(sm_top, sm_middle, sm_bottom)

                full_image[i-1] = re.sub(sm_top, r'\1\2O\3', full_image[i - 1])
                full_image[i] = re.sub(sm_middle, r'\1O\2OO\3OO\4OOO', full_image[i])
                full_image[i+1] = re.sub(sm_bottom, r'\1\2O\3O\4O\5O\6O\7O\8', full_image[i + 1])

                ret_val = True


    total_rough_seas = 0
    if ret_val:
        for y in range(len(full_image)):
            total_rough_seas += full_image[y].count('#')
            print(full_image[y])

    print(total_rough_seas)


    return ret_val

In [91]:
part2(part1(inputExample))

part 1 20899048083289
0
3 possible sea monster: #.O.##.OO#.#.OO.##.OOO##
  SEA MONSTER CONFIRMED!
^(..)(..................)#(.) ^(..)#(....)##(....)##(....)### ^(..)(.)#(..)#(..)#(..)#(..)#(..)#(...)
17 possible sea monster: .O##.#OO.###OO##..OOO##.
  SEA MONSTER CONFIRMED!
^(.)(..................)#(.) ^(.)#(....)##(....)##(....)### ^(.)(.)#(..)#(..)#(..)#(..)#(..)#(...)
21 possible sea monster: O....OO..#.OO####OOO..##
.####...#####..#...###..
#####..#..#.#.####..#.#.
.#.#...#.###...#.##.O#..
#.O.##.OO#.#.OO.##.OOO##
..#O.#O#.O##O..O.#O##.##
...#.#..##.##...#..#..##
#.##.#..#.#..#..##.#.#..
.###.##.....#...###.#...
#.####.#.#....##.#..#.#.
##...#..#....#..#...####
..#.##...###..#.#####..#
....#.##.#.#####....#...
..##.##.###.....#.##..#.
#...#...###..####....##.
.#.##...#.##.#.#.###...#
#.###.#..####...##..#...
#.###...#.##...#.##O###.
.O##.#OO.###OO##..OOO##.
..O#.O..O..O.#O##O##.###
#.#..##.########..#..##.
#.#####..#.#...##..#....
#....##..#.#########..##
#...#.....#..##...###.##
#

In [92]:
part2(part1(inputReal))

part 1 15006909892229
0
3 possible sea monster: #..#.##....#..........#.........#...#.....#.......O..##OO..#.OO..#.OOO...#.#...........#...##...
  SEA MONSTER CONFIRMED!
^(..................................................)(..................)#(.) ^(..................................................)#(....)##(....)##(....)### ^(..................................................)(.)#(..)#(..)#(..)#(..)#(..)#(...)
8 possible sea monster: #...#..##.....#.O..##OO....OO....OOO...#...##..##....#............#....#.......##.....#......#.#
  SEA MONSTER CONFIRMED!
^(................)(..................)#(.) ^(................)#(....)##(....)##(....)### ^(................)(.)#(..)#(..)#(..)#(..)#(..)#(...)
12 possible sea monster: ..............#..#..#...#..#...##..#.....#.#....#....#.#..#.##.##..#O#.#.OO..##OO....OOO.#.#...#
  SEA MONSTER CONFIRMED!
^(....................................................................)(..................)#(.) ^(.................................................